# Opacus를 활용한 Flower 연합학습 차등 개인정보보호

이번 튜토리얼에서는 개인정보보호 연합학습을 위해 PyTorch와 Flower를 사용해서 Opacus를 활용하는 방법을 알아보는 시간을 갖겠습니다.

아래의 코드들은 이전에 소개했던 사례들과는 다른 내용을 다루기 때문에, 이전 강의에서 다루었던 **주제**에 대해서만 간략하게 다룰 예정입니다.

연합학습은 본질적으로 원시 클라이언트 데이터 대신 모델 업데이트만 서버로 전송하여 개인정보를 보호하는 형식을 갖추고 있습니다.

그러나, 예를 들어서 훈련 데이터의 일부를 재구성하거나, 훈련된 모델에게만 주어진 특정 사용자의 구성원 자격을 추론할 수 있는 공격이 존재하는 것으로 밝혀졌습니다.

이러한 보안 위험을 완화하기 위해 사용되는 일반적인 접근 방식은 차등 개인정보보호입니다.

이번 튜토리얼에서는 차등 개인정보보호에 대한 개념을 소개하고 [Opacus library](https://opacus.ai/)를 사용하여 Flower 클라이언트에 추가하는 방법을 알려드리겠습니다.

## 차등 개인정보보호가 무엇인가요?
​
차등 개인정보보호란 모델의 파라미터와 임의의 측면 정보가 주어진 사용자에 대한 공격자의 학습 정보 위험에 대해 수학적으로 추론할 수 있는 개인정보보호에 대한 정의입니다.

차등 개인정보보호를 만족시키기 위한 훈련 알고리즘은 모델 훈련 과정 누락되거나 나타나는 단일 사용자가 모델에 너무 많은 영향을 미치지 않도록 해야 합니다.
​
형식적으로, ([Dwork et al. 2006](https://www.iacr.org/archive/eurocrypt2006/40040493/40040493.pdf))에 인접한 두 개의 데이터 세트 $d$ 와 $d'$가 주어지며,

여기서 단일 사용자에 속하는 $d$에서 모든 샘플을 추가하거나 제거함으로써 $d$로부터 $d'$를 생성할 수 있는

임의 훈련 알고리즘 $\mathcal{A}: \mathcal{D} \rightarrow \mathcal{M}$은 사용 가능한 훈련 데이터 세트를 사용 가능한 훈련 모델에 매핑하면$(\varepsilon, \delta)$,

차등 개인정보보호의 경우, 임의의 하위 데이터 세트 $O \in \mathcal{M}$에 대해 다음 수식을 얻을 수 있습니다.

$$Pr[\mathcal{A}(d) \in O] \leq e^\varepsilon Pr[\mathcal{A}(d') \in O] + \delta.$$

여기서 $\varepsilon$는 소위 개인정보보호 비용으로 정의할 수 있습니다.

낮을수록 개인정보보호 보장을 제공할 수 있습니다.

또 다른 파라미터인 $\delta$는 예기치 못한 상황이 발생하여 해당 보증을 이행할 수 없을 확률의 척도(낮을수록 좋음)으로 정의할 수 있습니다.

딥러닝의 경우 $(\varepsilon, \delta)$ 차등 개인정보보호를 달성하기 위해 현재 가장 일반적으로 사용되는 알고리즘은

DP-SGD ([Abadi et al. 2016](https://arxiv.org/pdf/1607.00133.pdf))로, FedAvg ([McMahan et al. 2017](https://arxiv.org/pdf/1710.06963v1.pdf))와 같은 연합학습 전략의 일부로 쉽게 사용할 수 있습니다.
​

DP-SGD에는 다음과 같이 사용자 개인정보보호를 향상시키는 데 사용되는 두 가지 주요 단계가 있습니다:​
​
- **Gradient norm clipping**은 전체 평균에 대한 단일 클라이언트의 영향을 보장하는 것으로, 모델과 데이터에 의해 결정되는 많은 요인 $L$의 최대 기울기 정규화로 제한합니다.

- **Gaussian noising**은 모델의 파라미터에 $\mathcal{N}(0, L^2\sigma^2)$를 추가하여 필요한 무작위성 요소를 삽입합니다(여기서 $\sigma$는 노이즈와 관련됨).
​

또한, 이 알고리즘은 임의로 균일하게 선택되는 클라이언트와 데이터 샘플에 의존하는 특성이 존재합니다.

특히 연합학습의 경우, 서버로 모델 파라미터를 전송하기 전에 모델 업데이트에 클라이언트 단에서 노이즈를 추가하거나 서버 단에서 노이즈를 추가할 수 있습니다.

중앙집중형 방식은 전체적으로 노이즈가 적기 때문에 보다 정확한 모델을 유도하지만, 신뢰할 수 있는 서버가 존재한다는 가정 하에 이루어집니다.

다음 예시는 클라이언트 단에서 차등 개인정보보호에 초점을 맞춥니다.

차등 개인정보보호 메커니즘을 제공하는 라이브러리의 또다른 중요한 부분은 개인정보보호 비용을 추적할 수 있는 방법을 제공하는 것입니다.

이는 동일한 데이터 세트에 대한 메커니즘의 여러 응용 프로그램이 제공 가능한 개인정보보호 보장을 변경하기 때문입니다.

데이터에 대한 액세스는 잠재적으로 클라이언트에 대한 더 많은 정보를 노출시키는 것을 의미합니다.

$\varepsilon$ 값과 경계를 계산하는 것은 수학적으로 매우 복잡하지만,

일반적으로 클라이언트에서 수행되는 모든 훈련 단계에 따라 점진적으로 $\varepsilon$가 증가하는 결과를 초래하는 반면,

연합학습 설정을 통한다면 모든 클라이언트 중에서 최대 $\varepsilon$가 전쳬 비용을 결정하는 병렬 구성의 형태를 띄게 될 것입니다.

## Opacus 설치

Opacus를 포함한 필요한 종속성을 설치하고 불러오는 것으로 시작해볼까요?

현재 Opacus의 출시 버전은 최신 버전의 PyTorch와 호환되지 않습니다.

따라서, 이전 버전의 `torch` 와 `torchvision`을 설치하겠습니다.

In [ ]:
!pip install torchcsprng==0.1.3+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install matplotlib opacus==0.14.0 torch==1.7.0 torchvision==0.8.0 git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]

from collections import OrderedDict
from typing import List

import flwr as fl
import numpy as np
import opacus                                           # <-- NEW
from opacus import PrivacyEngine                        # <-- NEW
from opacus.dp_model_inspector import DPModelInspector  # <-- NEW
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

print(torch.__version__)
print(torchvision.__version__)
print(opacus.__version__)

DEVICE = torch.device("cpu")
DEVICE = "cpu"  # Enable this line to force execution on CPU
print(f"Training on {DEVICE}")

## 작업 정의

이전과 마찬가지로 기본적인 작업을 정의하는 것으로 시작하겠습니다.

여기에는 데이터 불러오기 및 모델 구성과 `get_parameters`/`set_parameters`가 포함됩니다:

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 32

def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()


class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


print(len(trainloaders[0]))
print(len(valloaders[0]))
print(len(trainloaders[0].dataset))
print(len(valloaders[0].dataset))

## Opacus 예제
​
차등 개인정보보호를 보장하는 기계학습을 구현할 수 있는 여러 라이브러리가 있습니다.

이러한 것들은 일반적으로 클리핑 단계가 일반적으로 각 훈련 단계와 함께 수행되므로,

최적화를 위한 라이브러리별 래퍼를 가져야 하기 때문에 모델을 훈련하는 데 사용하는 라이브러리에 따라 달라지게 됩니다.

이번 예제는 PyTorch에 최적화된 Opacus를 사용하지만 [Tensorflow Privacy를 사용하여 DP-SGD를 구현하는 Flower 클라이언트](https://github.com/adap/flower/tree/main/examples/dp-sgd-mnist)도 있습니다.
​

또한 이번 예제는 [PyTorch Quickstart](https://flower.dev/docs/quickstart_pytorch.html)에 기반을 두고 있습니다(여러분들이 PyTorch에 익숙하고 코드도 다루실줄 안다고 믿고 있습니다). 
​
첫 번째 단계는 네트워크가 현재 모든 계층을 지원하지 않기 때문에 네트워크가 Opacus와 호환되는지 확인하는 것입니다(자세한 내용은 [문서](https://github.com/pytorch/opacus/blob/master/opacus/README.md)를 확인해보시길 바랍니다).

이를 위해 모델을 인스턴스화할 때 `DPModelInspector`를 사용합니다.

In [ ]:
def validate_model():
  model = Net()
  inspector = DPModelInspector()
  print(inspector.validate(model))


validate_model()

설정에서 가장 까다로운 부분은 개인정보보호 엔진에 대한 올바른 개인정보보호 파라미터를 찾는 것입니다.

- **Target delta $\delta$**: 훈련 데이터 세트의 크기의 역으로 설정해야 합니다. 예를 들어, 데이터 세트에 CIFAR10과 같은 50,000개의 훈련 데이터가 있는 경우, 이를 설정하기 좋은 수치는 $10^{-5}$ 입니다.
- **Noise multiplier $\sigma$**: 각 단계에서 추가된 노이즈의 양을 측정하면, $\varepsilon$의 값이 작을수록 커집니다.
- **Target epsilon $\varepsilon$**: 
고정된 **Noise multiplier**의 대안으로, 목표 $\varepsilon$가 주어지면 엔진에 의해 계산될 수 있습니다. 그러나 이 변수는 전역 훈련과 지역 훈련 라운드를 모두 고려해야 하기 때문에 연합학습 환경에서 알아내기 어려운 훈련 단계를 제공하도록 요구합니다.
- **Maximum gradient norm $L$**: 모델 설계, 클라이언트에 대한 훈련 데이터 양 및 학습 속도와 같은 요인에 크게 의존하는 이 파라미터의 경우, 너무 낮게 설정하면 높은 바이어스를 초래할 수 있고, 너무 높게 설정하면 모델 유틸리티가 손상될 수 있기 때문에 그리드 검색을 하는 것이 유용할 수 있습니다([Andrew et al. 2021](https://arxiv.org/pdf/1905.03871.pdf)).

샘플 비율 대신 `sample_rate = batch_size / sample_size`이므로, `batch_size`(한 단계에서 취한 훈련 샘플 수)와 `sample_size`(한 클라이언트의 데이터 세트 전체 크기)를 모두 제공할 수도 있습니다.

이 외에도 고급 파라미터를 지정할 수 있으며, [문서](https://opacus.ai/api/privacy_engine.html)와 [튜토리얼](https://opacus.ai/tutorials/)에서 더욱 자세한 정보를 찾을 수 있습니다.

이번 예제는 다음과 같은 파라미터를 사용하지만 최적화된 과정은 아닙니다(utility-privacy trade-off 측면에서 선호도에 따라 상이함).

In [ ]:
PARAMS = {
    'batch_size': 32,
    'train_split': 0.7,
    'local_epochs': 1
}
PRIVACY_PARAMS = {
    'target_delta': 1e-5,
    'noise_multiplier': 0.4,
    'max_grad_norm': 1.2
}

마지막 단계는 각 훈련에서 옵티마이저에 개인정보보호 엔진을 부착하고

선택적으로 지출된 현재 개인정보보호 비용을 받아 반환하는 것입니다:

In [ ]:
def train(net, trainloader, privacy_engine, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    # Attach privacy engine to optimizer
    privacy_engine.attach(optimizer)
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
    # Get privacy budget
    epsilon, _ = optimizer.privacy_engine.get_privacy_spent(PRIVACY_PARAMS['target_delta'])
    return epsilon


# Same as before
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

다음은 클라이언트의 `PrivacyEngine` 사례입니다:

In [ ]:
PE = {}

def get_privacy_engine(cid, model, sample_rate):
    if cid not in PE.keys():
        PE[cid] = PrivacyEngine(
            model,
            sample_rate = sample_rate,
            target_delta = PRIVACY_PARAMS['target_delta'],
            max_grad_norm = PRIVACY_PARAMS['max_grad_norm'],
            noise_multiplier = PRIVACY_PARAMS['noise_multiplier']
        )
    return PE[cid]  # Use the previously created PrivacyEngine

개인정보추적이 정확하지 않기 때문에 모델당 각 훈련 내내 유지되는 하나의 엔진을 보유하는 것이 중요합니다.

자원적으로 효울적인 단일 기기 시뮬레이션을 위해 Flower 가상 클라이언트 엔진을 사용하면서도 약간의 트릭을 써야합니다.

각 클라이언트에 대한 개인정보보호 엔진 초기화를 지연하고 딕셔너리에 참조를 유지해야 합니다.

이를 통해 가상 클라이언트 엔진을 사용 후에 `FlowerClient` 인스턴스를 삭제할 수 있지만,

여전히 각 클라이언트에 대해 매번 동일한 `PrivacyEngine` 인스턴스를 재사용하게 됩니다.

## FlowerClient 구현

그런 다음, 클라이언트의 훈련 함수에서 개인정보보호 비용을 사용자 지정 메트릭으로 반환할 수도 있습니다

([연합학습 집계 전략 오버라이딩](https://flower.dev/docs/saving-progress.html)에 의해 더 다양하게 사용할 수 있습니다).

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, privacy_engine) -> None:
        super().__init__()
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.privacy_engine = privacy_engine

    def get_parameters(self):
        return get_parameters(self.net)
    
    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        epsilon = train(self.net, self.trainloader, self.privacy_engine, PARAMS['local_epochs'])
        print(f"epsilon = {epsilon:.2f}")
        return get_parameters(self.net), len(self.trainloader), {"epsilon":epsilon}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

## 학습 시작

이젠 전에 하셨던 것처럼 `FlowerClient`를 사용할 수 있습니다!

In [ ]:
def client_fn(cid) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # PrivacyEngine
    sample_rate = BATCH_SIZE / len(trainloader.dataset) 
    pe = get_privacy_engine(cid, net, sample_rate)

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, pe)


# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_eval=1.0,  # Sample 100% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_eval_clients=10,  # Never sample less than 10 clients for evaluation
        min_available_clients=10,  # Wait until all 10 clients are available
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=5,
    strategy=strategy,
)

## 한계점 및 개선점
​
이와 같은 기본 설정을 구현하는 것은 비교적 간단하지만,

실제로 연합학습 환경에서 차등 개인정보보호 모델을 배치하는 것에 관해서는 많은 고려사항과 질문들이 있습니다.

위의 사항들은 궁극적으로 유용성과 개인정보보호 사이의 절충안을 고려하는 것으로 결론지을 수 있습니다.

한편, 차등 개인정보보호 모델은 수렴하는데 더욱 오랜 시간이 소요되며, 더 많은 계산이 필요하고([McMahan et al. 2017](https://arxiv.org/pdf/1710.06963v1.pdf)), 더 나쁜 모델을 초래할 수 있습니다([Bagdasaryan et al. 2019](https://proceedings.neurips.cc/paper/2019/file/fc0de4e0396fff257ea362983c2dda5a-Paper.pdf)).

반면에 상대적으로 큰 개인정보보호 비용은 사용자 개인정보보호에 유리합니다([Thakkar et al. 2020](https://arxiv.org/pdf/2006.07490.pdf)).

따라서 필요한 유틸리티 및 개인정보보호에 대한 개인정보보호 파라미터 및 가정을 고려하는 것이 중요합니다.

Flower에서 다음 단계는 기계 학습 라이브러리와 독립되도록 차등 개인정보보호를 적용한 방법으로 모델을 훈련시키는 방법을 찾는 것이지만,

​이 예시에서도 연합학습에서 개인정보보호를 실험하는 유용한 첫 번째 단계가 되기를 바랍니다.